# Crypto Trading System with Kraken API

A crypto trading system that fetches data from Kraken, visualizes price charts with Simple Moving Averages (SMA), uses LLM to analyze trading signals, and executes real trades on Kraken's sandbox environment.

## Features
- **Real-time Data**: Fetch BTC/USD and ETH/USD data from Kraken API
- **Technical Analysis**: Interactive charts with customizable SMA overlays
- **AI Trading Advisor**: LLM-powered trading recommendations based on technical indicators
- **Real Trading**: Execute actual trades on Kraken's sandbox environment
- **Multi-Model Support**: Choose from GPT-5, Claude Sonnet, or Gemini for analysis

## Setup Requirements
1. **API Keys**: Configure OpenAI, Anthropic, Google, and Kraken API keys in `.env` file
2. **Kraken Sandbox**: Get sandbox API credentials from [Kraken Developer Portal](https://www.kraken.com/features/api)
3. **Sandbox Funding**: Kraken sandbox may require initial funding for testing trades

## Process stes
1. **Market Data Tab**: Select crypto pair and SMA windows, fetch data
2. **Trading Analysis Tab**: Get LLM-powered trading recommendations
3. **Paper Trading Tab**: Execute real trades based on AI recommendations


In [ ]:
# Install dependencies
!uv add ccxt

In [ ]:
# Imports and Setup
import os
import time
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ccxt
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# API Key Setup
load_dotenv(override=True)

# LLM API Keys
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

# Kraken API Keys (for sandbox trading)
kraken_api_key = os.getenv('KRAKEN_API_KEY')
kraken_secret = os.getenv('KRAKEN_API_SECRET')

# Initialize API clients
anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

clients = {}
if openai_api_key:
    clients['openai'] = OpenAI(api_key=openai_api_key)
    print(f"✅ OpenAI API Key loaded: {openai_api_key[:8]}...")
else:
    print("❌ OpenAI API Key not set")

if anthropic_api_key:
    clients['anthropic'] = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
    print(f"✅ Anthropic API Key loaded: {anthropic_api_key[:7]}...")
else:
    print("❌ Anthropic API Key not set")

if google_api_key:
    clients['google'] = OpenAI(api_key=google_api_key, base_url=gemini_url)
    print(f"✅ Google API Key loaded: {google_api_key[:8]}...")
else:
    print("❌ Google API Key not set")

if kraken_api_key and kraken_secret:
    print(f"✅ Kraken API Keys loaded: {kraken_api_key[:8]}...")
else:
    print("❌ Kraken API Keys not set - paper trading will be disabled")

# Model configuration
MODELS = {
    "GPT-5": {"model_id": "gpt-5-mini", "provider": "openai"},
    "Claude Sonnet": {"model_id": "claude-sonnet-4-5-20250929", "provider": "anthropic"},
    "Gemini": {"model_id": "gemini-2.5-flash-lite", "provider": "google"}
}

CRYPTO_PAIRS = {
    "BTC/USD": "PI_XBTUSD",  # Bitcoin futures symbol
    "ETH/USD": "PI_ETHUSD"   # Ethereum futures symbol
}

print("✅ API setup complete!")


In [ ]:
# Kraken Data Fetcher Module
class KrakenDataFetcher:
    """Handles data fetching from Kraken API and SMA calculations"""
    
    def __init__(self):
        self.exchange = ccxt.kraken({
            'enableRateLimit': True,
            'timeout': 30000
        })
        self.cached_data = {}
    
    def fetch_ohlcv(self, symbol, timeframe='1h', limit=720):
        """Fetch OHLCV data from Kraken"""
        try:
            # Convert symbol to Kraken format
            # kraken_symbol = CRYPTO_PAIRS.get(symbol, symbol)
            kraken_symbol = symbol
            
            # Check cache first
            cache_key = f"{kraken_symbol}_{timeframe}_{limit}"
            if cache_key in self.cached_data:
                cache_time, data = self.cached_data[cache_key]
                if time.time() - cache_time < 300:  # 5 minute cache
                    print(f"📊 Using cached data for {symbol}")
                    return data
            
            print(f"🔄 Fetching {symbol} data from Kraken...")
            ohlcv = self.exchange.fetch_ohlcv(kraken_symbol, timeframe, limit=limit)
            
            # Convert to DataFrame
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            
            # Cache the data
            self.cached_data[cache_key] = (time.time(), df)
            
            print(f"✅ Fetched {len(df)} records for {symbol}")
            return df
            
        except Exception as e:
            print(f"❌ Error fetching data for {symbol}: {str(e)}")
            return pd.DataFrame()
    
    def calculate_sma(self, data, window):
        """Calculate Simple Moving Average"""
        if len(data) < window:
            return pd.Series(index=data.index, dtype=float)
        return data['close'].rolling(window=window).mean()
    
    def detect_crossover(self, sma_short, sma_long):
        """Detect SMA crossover signals"""
        if len(sma_short) < 2 or len(sma_long) < 2:
            return "No data"
        
        # Get the last two values
        short_current = sma_short.iloc[-1]
        short_previous = sma_short.iloc[-2]
        long_current = sma_long.iloc[-1]
        long_previous = sma_long.iloc[-2]
        
        # Check for crossover
        if pd.isna(short_current) or pd.isna(long_current) or pd.isna(short_previous) or pd.isna(long_previous):
            return "Insufficient data"
        
        # Golden cross: short SMA crosses above long SMA
        if short_previous <= long_previous and short_current > long_current:
            return "Golden Cross (BUY Signal)"
        
        # Death cross: short SMA crosses below long SMA
        if short_previous >= long_previous and short_current < long_current:
            return "Death Cross (SELL Signal)"
        
        # No crossover
        if short_current > long_current:
            return "Short SMA above Long SMA"
        else:
            return "Short SMA below Long SMA"
    
    def get_market_summary(self, data, sma_short, sma_long):
        """Get market summary for LLM analysis"""
        if data.empty:
            return {}
        
        current_price = data['close'].iloc[-1]
        price_change = ((current_price - data['close'].iloc[-2]) / data['close'].iloc[-2]) * 100 if len(data) > 1 else 0
        
        # Volume analysis
        avg_volume = data['volume'].rolling(window=10).mean().iloc[-1] if len(data) >= 10 else data['volume'].mean()
        current_volume = data['volume'].iloc[-1]
        volume_ratio = current_volume / avg_volume if avg_volume > 0 else 1
        
        # Price momentum (last 5 periods)
        momentum = ((data['close'].iloc[-1] - data['close'].iloc[-6]) / data['close'].iloc[-6]) * 100 if len(data) >= 6 else 0
        
        return {
            'current_price': current_price,
            'price_change_pct': round(price_change, 2),
            'current_volume': current_volume,
            'volume_ratio': round(volume_ratio, 2),
            'momentum_5d': round(momentum, 2),
            'sma_short': sma_short.iloc[-1] if not sma_short.empty else None,
            'sma_long': sma_long.iloc[-1] if not sma_long.empty else None,
            'crossover_status': self.detect_crossover(sma_short, sma_long)
        }

# Initialize data fetcher
data_fetcher = KrakenDataFetcher()
print("✅ Kraken Data Fetcher initialized!")


In [ ]:
# Chart Builder Module
class ChartBuilder:
    """Creates interactive charts with candlesticks and SMA overlays"""
    
    def __init__(self):
        self.colors = {
            'bullish': '#00ff88',
            'bearish': '#ff4444',
            'sma_short': '#2196F3',
            'sma_long': '#FF9800',
            'volume': '#9C27B0'
        }
    
    def create_candlestick_chart(self, data, sma_short, sma_long, symbol):
        """Create interactive candlestick chart with SMA overlays"""
        if data.empty:
            return go.Figure()
    
        try:
            # Create a simple figure without subplots first
            fig = go.Figure()
            
            # Add candlestick chart using only the data values
            fig.add_trace(go.Candlestick(
                x=list(range(len(data))), 
                open=data['open'].values,
                high=data['high'].values,
                low=data['low'].values,
                close=data['close'].values,
                name='Price',
                increasing_line_color='#00ff00',
                decreasing_line_color='#ff0000'
            ))
            
            # Add SMA lines if available
            if sma_short is not None and not sma_short.empty:
                # Filter out NaN values and align with data
                sma_short_clean = sma_short.dropna()
                if not sma_short_clean.empty:
                    sma_x = list(range(len(sma_short_clean)))
                    fig.add_trace(go.Scatter(
                        x=sma_x,
                        y=sma_short_clean.values,
                        mode='lines',
                        name='SMA Short',
                        line=dict(color='#2196F3', width=2)
                    ))
            
            if sma_long is not None and not sma_long.empty:
                # Filter out NaN values and align with data
                sma_long_clean = sma_long.dropna()
                if not sma_long_clean.empty:

                    sma_x = list(range(len(sma_long_clean)))
                    fig.add_trace(go.Scatter(
                        x=sma_x,
                        y=sma_long_clean.values,
                        mode='lines',
                        name='SMA Long',
                        line=dict(color='#FF9800', width=2)
                    ))
            
            # Update layout
            fig.update_layout(
                title=f'{symbol} Trading Chart',
                xaxis_title="Time Period",
                yaxis_title="Price ($)",
                height=600,
                showlegend=True,
                template='plotly_white'
            )
            
            return fig
            
        except Exception as e:
            print(f"❌ Error in chart creation: {str(e)}")
            # Return a simple empty figure
            return go.Figure()
    
    def _find_crossover_points(self, sma_short, sma_long, timestamps):
        """Find crossover points between SMAs"""
        if len(sma_short) < 2 or len(sma_long) < 2:
            return []
        
        crossover_points = []
        
        for i in range(1, len(sma_short)):
            if pd.isna(sma_short.iloc[i]) or pd.isna(sma_long.iloc[i]) or \
               pd.isna(sma_short.iloc[i-1]) or pd.isna(sma_long.iloc[i-1]):
                continue
            
            # Golden cross: short SMA crosses above long SMA
            if sma_short.iloc[i-1] <= sma_long.iloc[i-1] and sma_short.iloc[i] > sma_long.iloc[i]:
                crossover_points.append({
                    'timestamp': timestamps[i],
                    'type': 'Golden Cross'
                })
            
            # Death cross: short SMA crosses below long SMA
            elif sma_short.iloc[i-1] >= sma_long.iloc[i-1] and sma_short.iloc[i] < sma_long.iloc[i]:
                crossover_points.append({
                    'timestamp': timestamps[i],
                    'type': 'Death Cross'
                })
        
        return crossover_points
    
    def create_summary_chart(self, data, symbol):
        """Create a simple price summary chart"""
        if data.empty:
            return go.Figure()
        
        fig = go.Figure()
        
        # Add price line
        fig.add_trace(go.Scatter(
            x=data.index,
            y=data['close'],
            mode='lines',
            name='Price',
            line=dict(color='white', width=2)
        ))
        
        # Add volume as background bars
        fig.add_trace(go.Bar(
            x=data.index,
            y=data['volume'],
            name='Volume',
            opacity=0.3,
            yaxis='y2'
        ))
        
        fig.update_layout(
            title=f'{symbol} Price Summary',
            xaxis_title='Time',
            yaxis_title='Price (USD)',
            yaxis2=dict(title='Volume', overlaying='y', side='right'),
            template='plotly_dark',
            height=400
        )
        
        return fig

# Initialize chart builder
chart_builder = ChartBuilder()
print("✅ Chart Builder initialized!")


In [ ]:
# LLM Trading Advisor Module
class TradingAdvisor:
    """Uses LLM to analyze market data and provide trading recommendations"""
    
    def __init__(self):
        self.system_prompt = """You are an expert cryptocurrency trading advisor with deep knowledge of technical analysis and market dynamics. 

Your task is to analyze cryptocurrency market data and provide clear, actionable trading recommendations.

Key factors to consider:
1. SMA Crossover Signals: Golden Cross (short SMA > long SMA) suggests bullish momentum, Death Cross (short SMA < long SMA) suggests bearish momentum
2. Price Momentum: Recent price movements and trends
3. Volume Analysis: Volume spikes often indicate strong moves
4. Market Context: Overall market conditions and volatility

Provide your analysis in this exact format:
DECISION: [BUY/SELL/HOLD]
CONFIDENCE: [1-10 scale]
REASONING: [2-3 sentence explanation of your decision]
RISK_LEVEL: [LOW/MEDIUM/HIGH]

Be concise but thorough in your analysis. Focus on the most relevant technical indicators."""

    def analyze_market(self, market_summary, model_name="GPT-5", temperature=0.3):
        """Analyze market data and provide trading recommendation"""
        try:
            if not market_summary:
                return {
                    'decision': 'HOLD',
                    'confidence': 1,
                    'reasoning': 'Insufficient market data for analysis',
                    'risk_level': 'HIGH'
                }
            
            # Prepare context for LLM
            context = self._prepare_market_context(market_summary)
            
            # Query the selected model
            response = self._query_llm(model_name, context, temperature)
            
            # Parse the response
            recommendation = self._parse_llm_response(response)
            
            return recommendation
            
        except Exception as e:
            print(f"❌ Error in trading analysis: {str(e)}")
            return {
                'decision': 'HOLD',
                'confidence': 1,
                'reasoning': f'Analysis error: {str(e)}',
                'risk_level': 'HIGH'
            }
    
    def _prepare_market_context(self, market_summary):
        """Prepare market context for LLM analysis"""
        # Handle SMA values safely
        sma_short = market_summary.get('sma_short')
        sma_long = market_summary.get('sma_long')
        
        sma_short_str = f"${sma_short:.2f}" if sma_short is not None else "N/A"
        sma_long_str = f"${sma_long:.2f}" if sma_long is not None else "N/A"
        
        context = f"""
Current Market Analysis for Trading Decision:

PRICE DATA:
- Current Price: ${market_summary.get('current_price', 0):.2f}
- Price Change: {market_summary.get('price_change_pct', 0):.2f}%
- 5-Day Momentum: {market_summary.get('momentum_5d', 0):.2f}%

TECHNICAL INDICATORS:
- Short SMA: {sma_short_str}
- Long SMA: {sma_long_str}
- Crossover Status: {market_summary.get('crossover_status', 'Unknown')}

VOLUME ANALYSIS:
- Current Volume: {market_summary.get('current_volume', 0):,.0f}
- Volume Ratio: {market_summary.get('volume_ratio', 1):.2f}x (vs 10-day average)

Based on this data, provide a trading recommendation following the specified format.
"""
        return context
    
    def _query_llm(self, model_name, context, temperature):
        if model_name not in MODELS:
            raise ValueError(f"Unknown model: {model_name}")
        
        model_info = MODELS[model_name]
        provider = model_info['provider']
        model_id = model_info['model_id']
        
        if provider not in clients:
            raise ValueError(f"Client not available for {provider}")
        
        try:
            response = clients[provider].chat.completions.create(
                model=model_id,
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": context}
                ],
                temperature=temperature if model_id != "gpt-5-mini" else 1,
                max_completion_tokens=500  
            )
            
            return response.choices[0].message.content
            
        except Exception as e:
            raise Exception(f"LLM query failed: {str(e)}")

    
    def _parse_llm_response(self, response):
        """Parse LLM response into structured format"""
        try:
            lines = response.strip().split('\n')
            decision = 'HOLD'
            confidence = 5
            reasoning = 'No clear reasoning provided'
            risk_level = 'MEDIUM'
            
            for line in lines:
                line = line.strip()
                if line.startswith('DECISION:'):
                    decision = line.split(':', 1)[1].strip().upper()
                elif line.startswith('CONFIDENCE:'):
                    try:
                        confidence = int(line.split(':', 1)[1].strip())
                        confidence = max(1, min(10, confidence))  # Clamp between 1-10
                    except:
                        confidence = 5
                elif line.startswith('REASONING:'):
                    reasoning = line.split(':', 1)[1].strip()
                elif line.startswith('RISK_LEVEL:'):
                    risk_level = line.split(':', 1)[1].strip().upper()
            
            # Validate decision
            if decision not in ['BUY', 'SELL', 'HOLD']:
                decision = 'HOLD'
            
            # Validate risk level
            if risk_level not in ['LOW', 'MEDIUM', 'HIGH']:
                risk_level = 'MEDIUM'
            
            return {
                'decision': decision,
                'confidence': confidence,
                'reasoning': reasoning,
                'risk_level': risk_level,
                'raw_response': response
            }
            
        except Exception as e:
            return {
                'decision': 'HOLD',
                'confidence': 1,
                'reasoning': f'Failed to parse LLM response: {str(e)}',
                'risk_level': 'HIGH',
                'raw_response': response
            }
    
    def get_confidence_color(self, confidence):
        """Get color based on confidence level"""
        if confidence >= 8:
            return '#00ff88'  # Green
        elif confidence >= 6:
            return '#ffaa00'  # Orange
        else:
            return '#ff4444'  # Red
    
    def get_decision_emoji(self, decision):
        """Get emoji for decision"""
        emojis = {
            'BUY': '🟢',
            'SELL': '🔴',
            'HOLD': '🟡'
        }
        return emojis.get(decision, '❓')

# Initialize trading advisor
trading_advisor = TradingAdvisor()
print("✅ Trading Advisor initialized!")


In [ ]:
# Paper Trader Module for Kraken Sandbox
class PaperTrader:
    """Handles real trading on Kraken sandbox environment"""
    
    def __init__(self):
        self.exchange = None
        self.connected = False
        self.trade_history = []
        
        if kraken_api_key and kraken_secret:
            self._connect_sandbox()
    
    def _connect_sandbox(self):
        """Connect to Kraken sandbox environment"""
        try:
            self.exchange = ccxt.krakenfutures({
                'apiKey': kraken_api_key, 
                'secret': kraken_secret,
                'enableRateLimit': True,
            })

            self.exchange.set_sandbox_mode(True)
            
            # Test connection
            self.exchange.load_markets()
            self.connected = True
            print("✅ Connected to Kraken sandbox environment")
            
        except Exception as e:
            print(f"❌ Failed to connect to Kraken sandbox: {str(e)}")
            self.connected = False
    
    def get_balance(self):
        """Get real balance from Kraken sandbox"""
        if not self.connected:
            return {'USD': 0, 'BTC': 0, 'ETH': 0}
        
        try:
            # Fetch real balance from Kraken
            balance = self.exchange.fetch_balance()
            
            # Extract relevant currencies
            kraken_balance = {
                'USD': balance.get('USD', {}).get('free', 0),
                'BTC': balance.get('BTC', {}).get('free', 0),
                'ETH': balance.get('ETH', {}).get('free', 0)
            }
            
            return kraken_balance
            
        except Exception as e:
            print(f"❌ Error fetching balance: {str(e)}")
            return {'USD': 0, 'BTC': 0, 'ETH': 0}
    
    def place_order(self, symbol, side, amount, price=None):
        """Place a real trade order on Kraken sandbox"""
        if not self.connected:
            return {
                'success': False,
                'message': "❌ Not connected to Kraken sandbox",
                'trade': None
            }
        
        try:
            # Convert symbol to Kraken format
            kraken_symbol = CRYPTO_PAIRS.get(symbol, symbol)
            
            # Get current market price if not provided
            ticker = self.exchange.fetch_ticker(kraken_symbol)
            
            price = ticker["ask"] if price is None else price
            
            # Check balance before placing order
            balance = self.get_balance()
            crypto_symbol = symbol.split('/')[0]
            
            if side == 'buy':
                required_usd = amount / price
                if balance['USD'] < required_usd:
                    return {
                        'success': False,
                        'message': f"❌ Insufficient USD balance. Required: ${required_usd:.2f}, Available: ${balance['USD']:.2f}",
                        'trade': None
                    }
            else:  # sell                
                if  balance[crypto_symbol]  < (amount / price):
                    return {
                        'success': False,
                        'message': f"❌ Insufficient {crypto_symbol} balance. Required: {amount}, Available: {balance[crypto_symbol]}",
                        'trade': None
                    }
            
            # Place market order

            order = self.exchange.create_market_order(
                symbol=kraken_symbol,
                side=side,
                amount=amount
            )
            return {
                'success': True,
                'message': f"✅ Placed {side} order for {amount} {kraken_symbol}",
                'trade': order
            }
     
        except Exception as e:
            return {
                'success': False,
                'message': f"❌ Order failed: {str(e)}",
                'trade': None
            }

    def get_positions(self):
        """Get current positions from real balance"""
        try:
            balance = self.get_balance()
            positions = []
            
            for crypto, amount in balance.items():
                if crypto != 'USD' and amount > 0:
                    price = self._get_crypto_price(crypto)
                    positions.append({
                        'symbol': crypto,
                        'amount': amount,
                        'value_usd': amount * price
                    })
            
            return positions
            
        except Exception as e:
            print(f"❌ Error getting positions: {str(e)}")
            return []
    
    def _get_crypto_price(self, crypto):
        """Get current price for crypto"""
        try:
            symbol = f"{crypto}/USD"
            ticker = self.exchange.fetch_ticker(CRYPTO_PAIRS.get(symbol, symbol))
            return ticker['last']
        except:
            return 0
    
    def get_trade_history(self, limit=50):
        """Get real trade history from Kraken"""
        if not self.connected:
            return []
        
        try:
            # Fetch recent trades from Kraken
            trades = self.exchange.fetch_my_trades(limit=limit)
            print("Obtianed trades")
            print(trades)

            # Format trades for display
            formatted_trades = []
            for trade in trades:
                formatted_trades.append({
                    'timestamp': datetime.fromtimestamp(trade['timestamp'] / 1000),
                    'symbol': trade['symbol'],
                    'side': trade['side'],
                    'amount': trade['cost'],
                    'price': trade['price'],
                    'order_id': trade.get('order', 'unknown'),
                    'status': 'filled'
                })
            
            return formatted_trades
            
        except Exception as e:
            print(f"❌ Error fetching trade history: {str(e)}")
            return self.trade_history[-limit:] if self.trade_history else []
    
    def get_portfolio_value(self):
        """Calculate total portfolio value in USD from real balance"""
        try:
            balance = self.get_balance()
            total_value = balance['USD']
            
            for crypto, amount in balance.items():
                if crypto != 'USD' and amount > 0:
                    price = self._get_crypto_price(crypto)
                    total_value += amount * price
            
            return total_value
            
        except Exception as e:
            print(f"❌ Error calculating portfolio value: {str(e)}")
            return 0
    
    def get_order_status(self, order_id):
        """Get status of a specific order"""
        if not self.connected:
            return None
        
        try:
            order = self.exchange.fetch_order(order_id)
            return order
        except Exception as e:
            print(f"❌ Error fetching order status: {str(e)}")
            return None
    
    def cancel_order(self, order_id):
        """Cancel a pending order"""
        if not self.connected:
            return False
        
        try:
            result = self.exchange.cancel_order(order_id)
            return result
        except Exception as e:
            print(f"❌ Error canceling order: {str(e)}")
            return False

# Initialize paper trader
paper_trader = PaperTrader()


In [ ]:
# Global state variables
current_data = pd.DataFrame()
current_sma_short = pd.Series()
current_sma_long = pd.Series()
current_symbol = "BTC/USD"
current_market_summary = {}
current_recommendation = {}

# Gradio UI Functions
def fetch_market_data(symbol, sma_short_window, sma_long_window):
    """Fetch market data and calculate SMAs"""
    global current_data, current_sma_short, current_sma_long, current_symbol, current_market_summary
    
    try:
        current_symbol = symbol
        
        # Fetch data
        data = data_fetcher.fetch_ohlcv(symbol, '1h', 720)  # 30 days of hourly data
        
        if data.empty:
            return None, "❌ Failed to fetch market data", "No data available", "No data available"
        
        # Calculate SMAs
        sma_short = data_fetcher.calculate_sma(data, sma_short_window)
        sma_long = data_fetcher.calculate_sma(data, sma_long_window)
        
        # Store in global state
        current_data = data
        current_sma_short = sma_short
        current_sma_long = sma_long
        
        # Get market summary
        current_market_summary = data_fetcher.get_market_summary(data, sma_short, sma_long)
        
        # Create chart
        chart = chart_builder.create_candlestick_chart(data, sma_short, sma_long, symbol)
        
        # Prepare status message
        status = f"✅ Fetched {len(data)} records for {symbol}"
        status += f"\n📊 Current Price: ${current_market_summary.get('current_price', 0):.2f}"
        status += f"\n📈 Price Change: {current_market_summary.get('price_change_pct', 0):.2f}%"
        status += f"\n🔄 Crossover: {current_market_summary.get('crossover_status', 'Unknown')}"
        
        # Market summary for display
        sma_short = current_market_summary.get('sma_short')
        sma_long = current_market_summary.get('sma_long')
        
        sma_short_str = f"${sma_short:.2f}" if sma_short is not None else "N/A"
        sma_long_str = f"${sma_long:.2f}" if sma_long is not None else "N/A"
        
        summary = f"""
**Market Summary for {symbol}**
- Current Price: ${current_market_summary.get('current_price', 0):.2f}
- Price Change: {current_market_summary.get('price_change_pct', 0):.2f}%
- 5-Day Momentum: {current_market_summary.get('momentum_5d', 0):.2f}%
- Volume Ratio: {current_market_summary.get('volume_ratio', 1):.2f}x
- Short SMA: {sma_short_str}
- Long SMA: {sma_long_str}
- Crossover Status: {current_market_summary.get('crossover_status', 'Unknown')}
"""
        
        # Crossover status
        crossover = current_market_summary.get('crossover_status', 'Unknown')
        
        return chart, status, summary, crossover
        
    except Exception as e:
        return None, f"❌ Error: {str(e)}", "Error occurred", "Error"

def get_trading_recommendation(model_name, temperature):
    """Get LLM trading recommendation"""
    global current_market_summary, current_recommendation
    
    try:
        if not current_market_summary:
            return "❌ No market data available. Please fetch data first.", "No data", "No data", "No data"
        
        # Get recommendation from LLM
        recommendation = trading_advisor.analyze_market(current_market_summary, model_name, temperature)
        current_recommendation = recommendation
        
        # Format recommendation display
        decision_emoji = trading_advisor.get_decision_emoji(recommendation['decision'])
        confidence_color = trading_advisor.get_confidence_color(recommendation['confidence'])
        
        recommendation_text = f"""
**{decision_emoji} Trading Recommendation: {recommendation['decision']}**

**Confidence:** {recommendation['confidence']}/10
**Risk Level:** {recommendation['risk_level']}
**Reasoning:** {recommendation['reasoning']}

**Raw Analysis:**
{recommendation.get('raw_response', 'No raw response available')}
"""
        
        status = f"✅ Analysis complete using {model_name}"
        return status, recommendation_text, recommendation['decision'], recommendation['confidence'], recommendation['reasoning']
        
    except Exception as e:
        return f"❌ Error getting recommendation: {str(e)}", "ERROR", 0, "Error occurred"

def execute_trade(trade_action, trade_amount):
    """Execute paper trade"""
    global current_symbol, current_recommendation
    
    try:
        if not current_recommendation:
            return "❌ No trading recommendation available. Please get analysis first.", "No recommendation"
        
        if trade_action == "SKIP":
            return "⏭️ Trade skipped as requested", "Skipped"
        
        # Determine trade side
        if trade_action == "BUY" and current_recommendation['decision'] == 'BUY':
            side = 'buy'
        elif trade_action == "SELL" and current_recommendation['decision'] == 'SELL':
            side = 'sell'
        else:
            return f"⚠️ Trade action {trade_action} doesn't match recommendation {current_recommendation['decision']}", "Mismatch"
        
        # Execute trade
        result = paper_trader.place_order(current_symbol, side, trade_amount)
        
        print(result)

        if result['success']:
            return result['message'], "Success"
        else:
            return result['message'], "Failed"
            
    except Exception as e:
        return f"❌ Trade execution error: {str(e)}", "Error"

def get_portfolio_status():
    """Get current portfolio status"""
    try:
        balance = paper_trader.get_balance()
        portfolio_value = paper_trader.get_portfolio_value()
        positions = paper_trader.get_positions()
        trade_history = paper_trader.get_trade_history(10)
        
        # Format balance display
        balance_text = f"""
**Portfolio Balance:**
- USD: ${balance['USD']:.2f}
- BTC: {balance['BTC']:.6f}
- ETH: {balance['ETH']:.6f}
- **Total Value: ${portfolio_value:.2f}**
"""
        
        # Format positions
        if positions:
            positions_text = "**Current Positions:**\n"
            for pos in positions:
                positions_text += f"- {pos['symbol']}: {pos['amount']:.6f} (${pos['value_usd']:.2f})\n"
        else:
            positions_text = "**No current positions**"
        
        # Format trade history
        if trade_history:
            history_df = pd.DataFrame(trade_history)
            history_df['timestamp'] = pd.to_datetime(history_df['timestamp']).dt.strftime('%Y-%m-%d %H:%M')
            history_df = history_df[['timestamp', 'symbol', 'side', 'amount', 'price', 'status']]
        else:
            history_df = pd.DataFrame(columns=['timestamp', 'symbol', 'side', 'amount', 'price', 'status'])
        
        return balance_text, positions_text, history_df
        
    except Exception as e:
        return f"❌ Error getting portfolio: {str(e)}", "Error", pd.DataFrame()


In [ ]:
# Create Gradio Interface
def create_trading_interface():
    """Create the main Gradio interface with three tabs"""
    
    with gr.Blocks(title="Crypto Trading System", theme=gr.themes.Soft()) as interface:
        
        gr.Markdown("# 🚀 Crypto Trading System with AI Analysis")
        gr.Markdown("Fetch real-time crypto data, analyze with AI, and execute paper trades using Kraken API")
        
        # Status bar
        with gr.Row():
            api_status = gr.Textbox(
                label="API Status",
                value="✅ All systems ready" if clients else "❌ No API keys configured",
                interactive=False,
                scale=1
            )
            portfolio_status = gr.Textbox(
                label="Portfolio Status",
                value="Ready for trading",
                interactive=False,
                scale=2
            )
        
        # Tab 1: Market Data
        with gr.Tab("📊 Market Data"):
            gr.Markdown("### Fetch and analyze crypto market data")
            
            with gr.Row():
                with gr.Column(scale=2):
                    symbol_input = gr.Dropdown(
                        choices=list(CRYPTO_PAIRS.keys()),
                        value="BTC/USD",
                        label="Crypto Pair"
                    )
                    
                    with gr.Row():
                        sma_short_input = gr.Number(
                            value=20,
                            minimum=5,
                            maximum=100,
                            step=1,
                            label="Short SMA Window"
                        )
                        sma_long_input = gr.Number(
                            value=50,
                            minimum=10,
                            maximum=200,
                            step=1,
                            label="Long SMA Window"
                        )
                    
                    fetch_btn = gr.Button("📊 Fetch Market Data", variant="primary", size="lg")
                
                with gr.Column(scale=1):
                    market_status = gr.Textbox(
                        label="Fetch Status",
                        lines=4,
                        interactive=False
                    )
                    
                    crossover_status = gr.Textbox(
                        label="Crossover Status",
                        interactive=False
                    )
            
            # Chart display
            chart_output = gr.Plot(
                label="Price Chart with SMA Analysis",
                show_label=True
            )
            
            # Market summary
            market_summary = gr.Markdown(
                label="Market Summary",
                value="No data loaded"
            )
        
        # Tab 2: Trading Analysis
        with gr.Tab("🤖 AI Trading Analysis"):
            gr.Markdown("### Get AI-powered trading recommendations")
            
            with gr.Row():
                with gr.Column(scale=2):
                    analysis_model = gr.Dropdown(
                        choices=list(MODELS.keys()),
                        value=list(MODELS.keys())[0],
                        label="AI Model"
                    )
                    
                    analysis_temperature = gr.Slider(
                        minimum=0.0,
                        maximum=1.0,
                        value=0.3,
                        step=0.1,
                        label="Temperature (Lower = More Consistent)"
                    )
                    
                    analyze_btn = gr.Button("🤖 Get AI Recommendation", variant="primary", size="lg")
                
                with gr.Column(scale=1):
                    analysis_status = gr.Textbox(
                        label="Analysis Status",
                        lines=2,
                        interactive=False
                    )
            
            # Analysis results
            recommendation_display = gr.Markdown(
                label="Trading Recommendation",
                value="No analysis performed yet"
            )
            
            with gr.Row():
                decision_output = gr.Textbox(
                    label="Decision",
                    interactive=False
                )
                confidence_output = gr.Number(
                    label="Confidence (1-10)",
                    interactive=False
                )
                reasoning_output = gr.Textbox(
                    label="Reasoning",
                    lines=3,
                    interactive=False
                )
        
        # Tab 3: Paper Trading
        with gr.Tab("💰 Paper Trading"):
            gr.Markdown("### Execute paper trades based on AI recommendations")
            
            with gr.Row():
                with gr.Column(scale=2):
                    # Portfolio status
                    portfolio_balance = gr.Markdown(
                        label="Portfolio Balance",
                        value="Loading portfolio..."
                    )
                    
                    portfolio_positions = gr.Markdown(
                        label="Current Positions",
                        value="No positions"
                    )
                    
                    # Trade execution
                    trade_action = gr.Radio(
                        choices=["BUY", "SELL", "SKIP"],
                        value="SKIP",
                        label="Trade Action"
                    )
                    
                    trade_amount = gr.Number(
                        value=5,
                        minimum=1.0,
                        maximum=100.0,
                        step=1,
                        label="Trade Amount in USD (BTC/ETH)"
                    )
                    
                    execute_btn = gr.Button("💰 Execute Trade", variant="primary", size="lg")
                    
                    trade_result = gr.Textbox(
                        label="Trade Result",
                        lines=3,
                        interactive=False
                    )
                    trade_status = gr.Textbox(
                        label="Trade Status",
                        interactive=False,
                        visible=False  
                    )
                
                with gr.Column(scale=1):
                    # Trade history
                    trade_history = gr.Dataframe(
                        label="Recent Trades",
                        interactive=False,
                        wrap=True
                    )
                    
                    refresh_portfolio_btn = gr.Button("🔄 Refresh Portfolio", variant="secondary")
        
        # Event handlers
        fetch_btn.click(
            fetch_market_data,
            inputs=[symbol_input, sma_short_input, sma_long_input],
            outputs=[chart_output, market_status, market_summary, crossover_status]
        )
        
        analyze_btn.click(
            get_trading_recommendation,
            inputs=[analysis_model, analysis_temperature],
            outputs=[analysis_status, recommendation_display, decision_output, confidence_output, reasoning_output]
        )
        
        execute_btn.click(
            execute_trade,
            inputs=[trade_action, trade_amount],
            outputs=[trade_result, trade_status]
        )
        
        refresh_portfolio_btn.click(
            get_portfolio_status,
            outputs=[portfolio_balance, portfolio_positions, trade_history]
        )
        
        # Auto-refresh portfolio on load
        interface.load(
            get_portfolio_status,
            outputs=[portfolio_balance, portfolio_positions, trade_history]
        )
    
    return interface


In [ ]:
# Launch the Trading System
interface = create_trading_interface()

interface.launch(
    server_name="0.0.0.0",
    server_port=7860
)


## Testing and Validation

### Quick Test Functions

Run these cells to test individual components:


In [ ]:
# Test 1: Data Fetching
print("🧪 Testing Data Fetching...")
test_data = data_fetcher.fetch_ohlcv("BTC/USD", "1h", 24)  # Last 24 hours
if not test_data.empty:
    print(f"✅ Data fetch successful: {len(test_data)} records")
    print(f"   Latest price: ${test_data['close'].iloc[-1]:.2f}")
    
    # Test SMA calculation
    sma_20 = data_fetcher.calculate_sma(test_data, 20)
    sma_50 = data_fetcher.calculate_sma(test_data, 50)
    print(f"✅ SMA calculation successful")
    print(f"   SMA 20: ${sma_20.iloc[-1]:.2f}" if not sma_20.empty else "   SMA 20: N/A")
    print(f"   SMA 50: ${sma_50.iloc[-1]:.2f}" if not sma_50.empty else "   SMA 50: N/A")
    
    # Test crossover detection
    crossover = data_fetcher.detect_crossover(sma_20, sma_50)
    print(f"✅ Crossover detection: {crossover}")
else:
    print("❌ Data fetch failed")


In [ ]:
# Test 2: Chart Generation
print("\n🧪 Testing Chart Generation...")
if not test_data.empty:
    try:
        test_chart = chart_builder.create_candlestick_chart(test_data, sma_20, sma_50, "BTC/USD")
        print("✅ Chart generation successful")
        print(f"   Chart type: {type(test_chart)}")
        print(f"   Data points: {len(test_data)}")
    except Exception as e:
        print(f"❌ Chart generation failed: {str(e)}")
else:
    print("❌ No data available for chart testing")


In [ ]:
# Test 3: LLM Trading Analysis
print("\n🧪 Testing LLM Trading Analysis...")
if not test_data.empty and clients:
    try:
        # Get market summary
        market_summary = data_fetcher.get_market_summary(test_data, sma_20, sma_50)
        print(f"✅ Market summary generated: {len(market_summary)} fields")
        
        # Test LLM analysis (use first available model)
        available_models = [name for name, info in MODELS.items() if info['provider'] in clients]
        if available_models:
            model_name = available_models[0]
            print(f"   Testing with model: {model_name}")
            
            recommendation = trading_advisor.analyze_market(market_summary, model_name, 0.3)
            print(f"✅ LLM analysis successful")
            print(f"   Decision: {recommendation['decision']}")
            print(f"   Confidence: {recommendation['confidence']}/10")
            print(f"   Risk Level: {recommendation['risk_level']}")
        else:
            print("❌ No LLM models available")
    except Exception as e:
        print(f"❌ LLM analysis failed: {str(e)}")
else:
    print("❌ No data or LLM clients available for testing")


In [ ]:
# Test 4: Real Kraken Sandbox Trading
print("\n🧪 Testing Real Kraken Sandbox Trading...")
try:
    # Test portfolio status
    balance = paper_trader.get_balance()
    portfolio_value = paper_trader.get_portfolio_value()
    print(f"✅ Portfolio status retrieved from Kraken sandbox")
    print(f"   USD Balance: ${balance['USD']:.2f}")
    print(f"   BTC Balance: {balance['BTC']:.6f}")
    print(f"   ETH Balance: {balance['ETH']:.6f}")
    print(f"   Total Value: ${portfolio_value:.2f}")
    
    # Test real trade (very small amount to avoid issues)
    if balance['USD'] > 10:  # Only test if we have some USD balance
        test_result = paper_trader.place_order("BTC/USD", "sell", 5)
        if test_result['success']:
            print(f"✅ Real trade successful: {test_result['message']}")
            
            # Check updated balance
            new_balance = paper_trader.get_balance()
            print(f"   Updated USD: ${new_balance['USD']:.2f}")
            print(f"   Updated BTC: {new_balance['BTC']:.6f}")
        else:
            print(f"❌ Real trade failed: {test_result['message']}")
    else:
        print("⚠️ Insufficient USD balance for testing real trades")
        print("   Note: Kraken sandbox may require initial funding")
    
    # Test trade history
    trade_history = paper_trader.get_trade_history(5)
    print(f"✅ Trade history retrieved: {len(trade_history)} recent trades")
    
    # Test positions
    positions = paper_trader.get_positions()
    print(f"✅ Current positions: {len(positions)} active positions")
    for pos in positions:
        print(f"   {pos['symbol']}: {pos['amount']:.6f} (${pos['value_usd']:.2f})")
        
except Exception as e:
    print(f"❌ Real trading test failed: {str(e)}")

print("\n🎉 All tests completed!")
print("=" * 50)
